### Student Information
Name:Pheiroijam Prishika

Student ID:112062891

GitHub ID:Ph-Prishika

Kaggle name: prishika_matu

Kaggle private scoreboard snapshot:


 ![Screenshot from 2024-12-05 00-40-46.png](attachment:Screenshot from 2024-12-05 00-40-46.png)


---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home exercises** in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (0.6N + 1 - x) / (0.6N) * 10 + 20 points, where N is the total number of participants, and x is your rank. (ie. If there are 100 participants and you rank 3rd your score will be (0.6 * 100 + 1 - 3) / (0.6 * 100) * 10 + 20 = 29.67% out of 30%.)   
    Submit your last submission **BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developing the model for the competition (You can use code and comment on it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)__. 

In [ ]:
### Begin Assignment Here
mport pandas as pd
import re

import pandas as pd

# Load the datasets
tweets = pd.read_json('/kaggle/input/dm-2024-isa-5810-lab-2-homework/tweets_DM.json', lines=True)
labels = pd.read_csv('/kaggle/input/dm-2024-isa-5810-lab-2-homework/emotion.csv')
data_identification = pd.read_csv('/kaggle/input/dm-2024-isa-5810-lab-2-homework/data_identification.csv')


In [ ]:
# Display the first few rows of each dataset
print("Tweets Dataset:")
print(tweets.head())
print("\nLabels Dataset:")
print(labels.head())
print("\nData Identification Dataset:")
print(data_identification.head())

In [ ]:
def extract_tweet_details(tweets_df):
    # Extract nested fields
    tweets_df['tweet_id'] = tweets_df['_source'].apply(lambda x: x['tweet']['tweet_id'])
    tweets_df['text'] = tweets_df['_source'].apply(lambda x: x['tweet']['text'])
    tweets_df['hashtags'] = tweets_df['_source'].apply(lambda x: x['tweet']['hashtags'])
    return tweets_df.drop(columns=['_source', '_index', '_type', '_crawldate', '_score'])

def merge_data(tweets, labels, split_info):
    # Merge tweets with labels and split information
    merged = tweets.merge(labels, on='tweet_id', how='left')
    merged = merged.merge(split_info, on='tweet_id', how='left')
    return merged

def main():
    # Load datasets
    tweets, labels, split_info = load_data()
    tweets = extract_tweet_details(tweets)
    
    # Merge datasets
    merged_data = merge_data(tweets, labels, split_info)
    
    # Display the first 10 entries of the merged dataframe in a styled table
    print("First 10 entries of the merged data:")
    styled_merged_data = merged_data.head(10).style.set_properties(**{
        'border': '1px solid black',
        'text-align': 'left',
        'background-color': '#f9f9f9',
        'padding': '5px'
    }).set_caption("Merged Dataset: First 10 Entries").set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#f4f4f4'), ('font-weight', 'bold')]}
    ])
    display(styled_merged_data)

if __name__ == "__main__":
    main()

In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk

# Download necessary NLTK data
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def clean_text_with_stemming_and_lemmatization(text):
    # Remove emojis, special symbols, and non-alphanumeric characters
    text = re.sub("["
                  u"\U0001F600-\U0001F64F"
                  u"\U0001F300-\U0001F5FF"
                  u"\U0001F680-\U0001F6FF"
                  u"\U0001F700-\U0001F77F"
                  u"\U0001F780-\U0001F7FF"
                  u"\U0001F800-\U0001F8FF"
                  u"\U0001F900-\U0001F9FF"
                  u"\U0001FA00-\U0001FA6F"
                  u"\U0001FA70-\U0001FAFF"
                  u"\U00002702-\U000027B0"
                  u"\U000024C2-\U0001F251"
                  "]+", '', text, flags=re.UNICODE)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    
    # Apply stemming and lemmatization
    processed_words = []
    for word in words:
        stemmed_word = stemmer.stem(word)
        lemmatized_word = lemmatizer.lemmatize(stemmed_word)
        processed_words.append(lemmatized_word)
    
    # Join the processed words back into a single string
    processed_text = ' '.join(processed_words)
    return processed_text

# Load and preprocess the data
def load_and_preprocess_data():
    # Load datasets
    tweets = pd.read_json('/kaggle/input/dm-2024-isa-5810-lab-2-homework/tweets_DM.json', lines=True)
    tweets['_source'] = tweets['_source'].apply(lambda x: x['tweet'])
    tweets = pd.concat([tweets.drop('_source', axis=1), pd.json_normalize(tweets['_source'])], axis=1)
    labels = pd.read_csv('/kaggle/input/dm-2024-isa-5810-lab-2-homework/emotion.csv')
    data_identification = pd.read_csv('/kaggle/input/dm-2024-isa-5810-lab-2-homework/data_identification.csv')
    
    # Merge datasets
    tweets = tweets.merge(labels, on='tweet_id', how='left')
    tweets = tweets.merge(data_identification, on='tweet_id', how='left')
    
    # Clean and process text
    tweets['text'] = tweets['text'].apply(clean_text_with_stemming_and_lemmatization)
    return tweets

# Load and preprocess data
tweets = load_and_preprocess_data()

# Split the data into train and test sets
train_data = tweets[tweets['identification'] == 'train']
test_data = tweets[tweets['identification'] == 'test']

# Drop unnecessary columns
columns_to_drop = ['_type', 'hashtags', '_crawldate']
train_data = train_data.drop(columns=columns_to_drop, errors='ignore')
test_data = test_data.drop(columns=columns_to_drop + ['emotion'], errors='ignore')


# Save and display datasets
def save_and_display_datasets(merged_data):
    # Separate into train and test datasets
    train_data = merged_data[merged_data['identification'] == 'train']
    test_data = merged_data[merged_data['identification'] == 'test']
    
    # Save to CSV
    train_data.to_csv('/kaggle/working/train_dataset.csv', index=False)
    test_data.to_csv('/kaggle/working/test_dataset.csv', index=False)

    print("Datasets saved as train_dataset.csv and test_dataset.csv")

    # Display first 10 rows of train and test datasets as styled tables
    print("Train Dataset (First 10 Entries):")
    display(train_data.head(10).style.set_caption("Train Dataset").set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#d9edf7'), ('font-weight', 'bold')]}]))
    
    print("Test Dataset (First 10 Entries):")
    display(test_data.head(10).style.set_caption("Test Dataset").set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#f2dede'), ('font-weight', 'bold')]}]))

def main():
    # Load datasets
    tweets, labels, split_info = load_data()
    
    # Extract tweet details and remove emojis
    tweets = extract_tweet_details(tweets)
    
    # Merge datasets
    merged_data = merge_data(tweets, labels, split_info)
    
    # Save and display datasets
    save_and_display_datasets(merged_data)

if __name__ == "__main__":
    main()

In [ ]:
# Load the datasets if they are not already in memory
train_data = pd.read_csv('/kaggle/working/train_data.csv')
test_data = pd.read_csv('/kaggle/working/test_data.csv')

# Remove the '_score' column from both datasets
train_data = train_data.drop(columns=['_score','_index'], errors='ignore')
test_data = test_data.drop(columns=['_score','_index'], errors='ignore')

# Save the modified datasets back to CSV files
train_data.to_csv('/kaggle/working/train_data.csv', index=False)
test_data.to_csv('/kaggle/working/test_data.csv', index=False)

# Print a few rows to confirm the column is removed
print("Updated Training Data:")
print(train_data.head())
print("\nUpdated Testing Data:")
print(test_data.head())

In [ ]:

# Assuming train_data and test_data are already loaded and available
# If not, load or regenerate them as per previous steps

# Save the training data
train_data.to_csv('/kaggle/working/train_data.csv', index=False, sep=',', quotechar='"', quoting=csv.QUOTE_ALL)

# Save the testing data (assuming 'emotion' column has been removed)
test_data.to_csv('/kaggle/working/test_data.csv', index=False, sep=',', quotechar='"', quoting=csv.QUOTE_ALL)


# BERT

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from transformers import AdamW
from tqdm import tqdm
import numpy as np



# Load the training and testing datasets
train_data = pd.read_csv('/kaggle/working/train_data.csv')
test_data = pd.read_csv('/kaggle/working/test_data.csv')



In [ ]:
label_encoder = LabelEncoder()
train_data['emotion'] = label_encoder.fit_transform(train_data['emotion'])

print("Training Data Sample:")
print(train_data.head())
print("\nTest Data Sample:")
print(test_data.head())
class TweetDataset(Dataset):
    def __init__(self, data, tokenizer, max_len, labels=True):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['text']
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            padding="max_length",
            return_attention_mask=True,
            return_tensors="pt"
        )
        item = {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0)
        }
        if self.labels:
            item['label'] = torch.tensor(self.data.iloc[idx]['emotion'], dtype=torch.long)
        return item
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(train_data['emotion'].unique())
).to(device)

In [ ]:
# Hyperparameters
MAX_LEN = 128
BATCH_SIZE = 32

In [ ]:
# Create datasets
train_dataset = TweetDataset(train_data, tokenizer, MAX_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = TweetDataset(test_data, tokenizer, MAX_LEN, labels=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

In [ ]:
# Training loop
EPOCHS = 2
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f"Loss: {avg_loss:.4f}")
model.save_pretrained('bert_model')
tokenizer.save_pretrained('bert_model')
print("Predicting...")
model.eval()
predictions = []
tweet_ids = test_data['tweet_id'].tolist()

In [ ]:
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        batch_predictions = torch.argmax(logits, dim=-1).cpu().numpy()
        predictions.extend(batch_predictions)

In [ ]:
# Decode predictions to original emotion labels
predicted_labels = label_encoder.inverse_transform(predictions)
# Create the submission file
submission = pd.DataFrame({'id': tweet_ids, 'emotion': predicted_labels})
submission.to_csv('Submission.csv', index=False)
print("Submission file created successfully!")